## Segmenting and Clustering in Toronto part 2

### Utilize Foursquare location data

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# download url data from Wikipedia

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
canada_data = BeautifulSoup(source, 'lxml')
table = canada_data.find('table')

In [3]:
# creat a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [4]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [5]:
df = df[df['Borough'] != 'Not assigned']

print(df['Borough'].unique())

df = df.reset_index(drop=True)

df.head()

['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Drop duplicate rows

In [6]:
df.drop_duplicates(inplace=True)

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
for row in range(len(df.index)):
    if(df.loc[row,'Neighborhood']=='Not assigned'):
        print(df.loc[row,:])
        df[toronto_df.loc[row,'Neighborhood']=='Not assigned']=df.loc[row,'Borough']     

#### Print dataframe's dimensions using .shape

In [8]:
df.shape

(103, 3)

#### Retrieve longitude and latitude coordinates for each postal code

Now a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name is build, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code.

To that, I will use the geospatial data from the link below which points to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Firstly, let's define the function **get_ccords(postal_code)** that takes as input the postal code returns the geographical coordinates for each postal code (for the sake of the assignemnt).

In [9]:
import geocoder

def get_coords(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

#### Read geospatial data

In [10]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [11]:
geo_df.shape

(103, 3)

In [12]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df, on='Postalcode')


In [13]:
geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


Re-order the columns so as they displayed in the following order **Postalcode, Borough, Neighborhood, Latitude, Longitude**

In [14]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
